In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd
from os.path import join, basename

sys.path.append(str(Path(getcwd()).parent))

In [ ]:
import numpy as np
import pandas as pd
from unidecode import unidecode
from datetime import date, timedelta, datetime
from os.path import dirname, join, isfile, isdir
# from bulletin.utils.functions import faixa_etaria_linhas, faixa_etaria_colunas
from bulletin import default_input, default_output

In [ ]:
def faixas_etarias_template(casos, faixa_etaria, faixa_etaria_labels):
    faixa_etaria_index = dict(enumerate(faixa_etaria_labels))
    index_casos = np.digitize(casos['idade_original'], faixa_etaria, right=False)
    casos['faixa_etaria'] = [ index for index in index_casos ]
    obitos = casos.loc[ casos['data_obito'].notnull() ].copy()

    faixa_etaria_casos = casos.groupby(by=['faixa_etaria'])[['sexo']].count().reset_index().rename(columns={'sexo':'quantidade'})

    faixa_etaria_obitos = obitos.groupby(by=['faixa_etaria'])[['sexo']].count().reset_index().rename(columns={'sexo':'quantidade'})
    
    return faixa_etaria_casos, faixa_etaria_obitos

In [ ]:
def faixa_etaria_linhas(casos, file_name, faixa_etaria, faixa_etaria_labels):
    faixa_etaria_index = dict(enumerate(faixa_etaria_labels))
    index_casos = np.digitize(casos['idade_original'], faixa_etaria, right=False)

    casos['faixa_etaria'] = [ index for index in index_casos ]

    casos['mes_caso'] = casos.apply(lambda row: row['data_diagnostico'].strftime('%m'), axis=1)
    casos['ano_caso'] = casos.apply(lambda row: row['data_diagnostico'].strftime('%Y'), axis=1)


    obitos = casos.loc[ casos['data_obito'].notnull() ].copy()
    obitos['mes_obito'] = obitos.apply(lambda row: row['data_obito'].strftime('%m'), axis=1)
    obitos['ano_obito'] = obitos.apply(lambda row: row['data_obito'].strftime('%Y'), axis=1)



    faixa_etaria_casos = casos.groupby(by=['ano_caso','mes_caso','faixa_etaria'])[['sexo','ibge_res_pr']].count().rename(columns={'sexo':'0','ibge_res_pr':'1'})
    faixa_etaria_casos = faixa_etaria_casos.unstack()
    faixa_etaria_casos = faixa_etaria_casos.swaplevel(1,0,1).sort_index(1).rename(columns={'0':'qtde','1':'%'})

    faixa_etaria_casos.columns = pd.MultiIndex.from_tuples(faixa_etaria_casos.columns)
    faixa_etaria_casos['total'] = faixa_etaria_casos.xs('qtde',level=1, axis=1).sum(axis=1)


    faixa_etaria_casos = faixa_etaria_casos.sort_index()

    for ano_mes in faixa_etaria_casos.index:
        for faixa_etaria, tipo in faixa_etaria_casos.columns:
            if tipo == '%':
#                 print(faixa_etaria_casos.loc[ano_mes,'total'])
                faixa_etaria_casos.loc[ano_mes,(faixa_etaria,'%')] = faixa_etaria_casos.loc[ano_mes,(faixa_etaria,'%')] / faixa_etaria_casos.loc[ano_mes,'total']

    faixa_etaria_casos.columns = faixa_etaria_casos.columns.set_levels([ faixa_etaria_index[index] if isinstance(index,int) else index for index in faixa_etaria_casos.columns.levels[0]],0,False)





    faixa_etaria_obitos = obitos.groupby(by=['ano_obito','mes_obito','faixa_etaria'])[['sexo','ibge_res_pr']].count().rename(columns={'sexo':'0','ibge_res_pr':'1'})
    faixa_etaria_obitos = faixa_etaria_obitos.unstack()
    faixa_etaria_obitos = faixa_etaria_obitos.swaplevel(1,0,1).sort_index(1).rename(columns={'0':'qtde','1':'%'})

    faixa_etaria_obitos.columns = pd.MultiIndex.from_tuples(faixa_etaria_obitos.columns)
    faixa_etaria_obitos['total'] = faixa_etaria_obitos.xs('qtde',level=1, axis=1).sum(axis=1)

    faixa_etaria_obitos = faixa_etaria_obitos.sort_index()

    for ano_mes in faixa_etaria_obitos.index:
        for faixa_etaria, tipo in faixa_etaria_obitos.columns:
            if tipo == '%':
#                 print(faixa_etaria_obitos.loc[ano_mes,'total'])
                faixa_etaria_obitos.loc[ano_mes,(faixa_etaria,'%')] = faixa_etaria_obitos.loc[ano_mes,(faixa_etaria,'%')] / faixa_etaria_obitos.loc[ano_mes,'total']
                
    faixa_etaria_obitos.columns = faixa_etaria_obitos.columns.set_levels([ faixa_etaria_index[index] if isinstance(index,int) else index for index in faixa_etaria_obitos.columns.levels[0]],0,False)


    writer = pd.ExcelWriter(file_name,
                            engine='xlsxwriter',
                            datetime_format='dd/mm/yyyy',
                            date_format='dd/mm/yyyy')

    workbook = writer.book

    faixa_etaria_casos.to_excel(writer,sheet_name=f"faixa_etaria_casos")
    worksheet = writer.sheets[f"faixa_etaria_casos"]


    faixa_etaria_obitos.to_excel(writer,sheet_name=f"faixa_etaria_obitos")
    worksheet = writer.sheets[f"faixa_etaria_obitos"]

    writer.save()

In [ ]:
def normalize_number(num, cast=int, error='fill', fill='-1'):
    try:
        return cast(num)
    except ValueError:
        if error == 'raise':
            raise Exception(ValueError)
        elif error == 'fill':
            return normalize_number(fill, cast, 'raise')


def trim_overspace(string):
    parts = filter(lambda part: len(part) > 0, string.split(" "))
    return " ".join(parts)


def normalize_labels(label):
    label = str(label).replace("'", " ").replace(".", "").replace("\n", "").replace(",", "").lower()
    label = trim_overspace(label).replace(" ", "_")
    label = unidecode(label)

    return label


def normalize_text(txt):
    if not txt is None:
        txt = txt.upper()
        txt = unidecode(txt)
        txt = trim_overspace(txt)
        txt =  "".join([x if ('A' <= x <= 'Z') or (x in [' ','-','_','/','\'']) else '' for x in txt])
    else:
        txt = ''
    
    return txt

In [ ]:
hoje = datetime.today()
ontem = hoje - timedelta(1)
anteontem = ontem - timedelta(1)

day_chosen = hoje

pd.set_option('display.max_columns', None)
pd.set_option("display.precision", 5)

meses = ['janeiro','fevereiro','marco','abril','maio','junho','julho','agosto','setembro','outubro','novembro','dezembro']

In [ ]:
df_casos = pd.read_csv(join(default_output, 'comunicacao', f"INFORME_EPIDEMIOLOGICO_{str(day_chosen.day).zfill(2)}_{str(day_chosen.month).zfill(2)}_{day_chosen.year}_Geral.csv"), sep=';', converters={'IDADE_ORIGINAL': lambda x: normalize_number(x,fill=0)}, parse_dates=['DATA_DIAGNOSTICO','DATA_OBITO'], date_parser=lambda x: pd.to_datetime(x, format='%Y-%m-%d'))

df_casos.columns = [ normalize_labels(x) for x in df_casos.columns ]
df_casos

In [ ]:
# DADOS DO PARANÁ
df_casos = df_casos.loc[df_casos['uf_residencia'] == 'PR']

In [ ]:
# DADOS DE CURITIBA
# casos = casos.loc[casos["ibge_res_pr"]== 4106902]

In [ ]:
# DADOS DE REGIONAIS DE SAÚDE


# 05ª RS - Guarapuava
# list_IBGE_RS = [4103040, 4103958, 4104428, 4104451, 4108452, 4108650, 4109401, 4113254, 4113304, 4115457, 4117057, 4117800, 4119301, 4119608, 4120150, 4120606, 4121752, 4122156, 4127965, 4128658]

# 12ª RS - Umuarama
# list_IBGE_RS = [4128625,4100707,4100509,4103370,4103479,4106605,4107256,4107520,4108320,4109906,4110607,4111555,4114708,4115101,4117206,4118857,4118907,4125357,4126900,4128104,4128807]

# casos_RS = pd.DataFrame()
# for i in list_IBGE_RS:
#     casos_temp = casos.loc[casos["ibge_res_pr"] == i]
#     casos_RS = casos_RS.append(casos_temp)
# casos = casos_RS

In [ ]:
#--------------------------------------- APRESENTAÇÃO CURTA

# FAIXA ETÁRIA 4:
casos = df_casos.copy()
file_name = join(default_output, 'dash', 'RAW_faixa_etaria_por_mes(4).xlsx')
faixa_etaria = [10,20]
faixa_etaria_labels = ['0-9','10-19','>=20']
faixa_etaria_linhas(casos, file_name, faixa_etaria, faixa_etaria_labels)
# faixa_etaria_colunas(casos, file_name, faixa_etaria, faixa_etaria_labels)



# FAIXA ETÁRIA 10:
casos = df_casos.copy()
file_name = join(default_output, 'dash', 'RAW_faixa_etaria_por_mes(10).xlsx')
faixa_etaria = [10,20,30,40,50,60]
faixa_etaria_labels = ['0-9','10-19','20-29','30-39','40-49','50-59','>=60']
faixa_etaria_linhas(casos, file_name, faixa_etaria, faixa_etaria_labels)
# faixa_etaria_colunas(casos, file_name, faixa_etaria, faixa_etaria_labels)

In [ ]:
casos = df_casos.copy()
faixa_etaria = [10,20,40,60,np.inf]
faixa_etaria_labels = ['<10','10-19','20-39','40-59','>=60']
casos_template, obitos_template = faixas_etarias_template(casos, faixa_etaria, faixa_etaria_labels)
casos_template.to_csv(join(default_output, 'dash', 'casos_pr_template.csv'), index=False, sep=';', encoding='utf-8-sig')
obitos_template.to_csv(join(default_output, 'dash', 'obitos_pr_template.csv'), index=False, sep=';', encoding='utf-8-sig')

In [ ]:
casos = df_casos.copy()
casos = casos.loc[casos['ibge_res_pr'] == 4106902]
faixa_etaria = [10,20,40,60,np.inf]
faixa_etaria_labels = ['<10','10-19','20-39','40-59','>=60']
casos_template, obitos_template = faixas_etarias_template(casos, faixa_etaria, faixa_etaria_labels)
casos_template.to_csv(join(default_output, 'dash', 'casos_curitiba_template.csv'), index=False, sep=';', encoding='utf-8-sig')
obitos_template.to_csv(join(default_output, 'dash', 'obitos_curitiba_template.csv'), index=False, sep=';', encoding='utf-8-sig')

In [ ]:
# FAIXA ETÁRIA CRIANÇAS E ADOLESCENTES:
# casos = df_casos.copy()
# file_name = join(default_output, 'dash', 'RAW_faixa_etaria_por_mes(criancas_adolescentes).xlsx')
# faixa_etaria = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
# faixa_etaria_labels = ['<=1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','>=18']
# faixa_etaria_linhas(casos, file_name, faixa_etaria, faixa_etaria_labels)
# faixa_etaria_colunas(casos, file_name, faixa_etaria, faixa_etaria_labels)

In [ ]:
#--------------------------------------- APRESENTAÇÃO LONGA

# # FAIXA ETÁRIA:
# file_name = join(default_output, 'dash', 'RAW_faixa_etaria_por_mes.xlsx')
# faixa_etaria = [60,70,80,90,100,np.inf]
# faixa_etaria_labels = ['<=59','60-69','70-79','80-89','90-99','>=100']
# faixa_etaria_linhas(casos, file_name, faixa_etaria, faixa_etaria_labels)
# # faixa_etaria_colunas(casos, file_name, faixa_etaria, faixa_etaria_labels)


# # FAIXA ETÁRIA 2:
# file_name = join(default_output, 'dash', 'RAW_faixa_etaria_por_mes(2).xlsx')
# faixa_etaria = [10,20,40,60,np.inf]
# faixa_etaria_labels = ['<10','10-19','20-39','40-59','>=60']
# faixa_etaria_linhas(casos, file_name, faixa_etaria, faixa_etaria_labels)
# # faixa_etaria_colunas(casos, file_name, faixa_etaria, faixa_etaria_labels)


# # FAIXA ETÁRIA 3:
# file_name = join(default_output, 'dash', 'RAW_faixa_etaria_por_mes(3).xlsx')
# faixa_etaria = [6,10,20,30,40,50,60,70,80]
# faixa_etaria_labels = ['00-05','06-09','10-19','20-29','30-39','40-49','50-59','60-69','70-79','80 e mais']
# faixa_etaria_linhas(casos, file_name, faixa_etaria, faixa_etaria_labels)
# # faixa_etaria_colunas(casos, file_name, faixa_etaria, faixa_etaria_labels)


# # FAIXA ETÁRIA 4:
# file_name = join(default_output, 'dash', 'RAW_faixa_etaria_por_mes(4).xlsx')
# faixa_etaria = [10,20]
# faixa_etaria_labels = ['0-9','10-19','>=20']
# faixa_etaria_linhas(casos, file_name, faixa_etaria, faixa_etaria_labels)
# # faixa_etaria_colunas(casos, file_name, faixa_etaria, faixa_etaria_labels)


# # FAIXA ETÁRIA 5:
# file_name = join(default_output, 'dash', 'RAW_faixa_etaria_por_mes(5).xlsx')
# faixa_etaria = [80,85,90]
# faixa_etaria_labels = ['<80','80-84','85-89','>=90']
# faixa_etaria_linhas(casos, file_name, faixa_etaria, faixa_etaria_labels)
# # faixa_etaria_colunas(casos, file_name, faixa_etaria, faixa_etaria_labels)


# # FAIXA ETÁRIA 7:
# file_name = join(default_output, 'dash', 'RAW_faixa_etaria_por_mes(7).xlsx')
# faixa_etaria = [60,70,80]
# faixa_etaria_labels = ['<60','60-69','70-79','>=80']
# faixa_etaria_linhas(casos, file_name, faixa_etaria, faixa_etaria_labels)
# # faixa_etaria_colunas(casos, file_name, faixa_etaria, faixa_etaria_labels)


# # FAIXA ETÁRIA 9:
# file_name = join(default_output, 'dash', 'RAW_faixa_etaria_por_mes(9).xlsx')
# faixa_etaria = [60,70,80,90]
# faixa_etaria_labels = ['<60','60-69','70-79','80-89','>=90']
# faixa_etaria_linhas(casos, file_name, faixa_etaria, faixa_etaria_labels)


# # FAIXA ETÁRIA 10:
# file_name = join(default_output, 'dash', 'RAW_faixa_etaria_por_mes(10).xlsx')
# faixa_etaria = [10,20,30,40,50,60,np.inf]
# faixa_etaria_labels = ['0-9','10-19','20-29','30-39','40-49','50-59','>=60']
# faixa_etaria_linhas(casos, file_name, faixa_etaria, faixa_etaria_labels)
# # faixa_etaria_colunas(casos, file_name, faixa_etaria, faixa_etaria_labels)

In [ ]:
# FAIXAS ETÁRIAS DE IDOSOS:

# # FAIXA ETÁRIA 8:
# casos = df_casos.copy()
# file_name = join(default_output, 'dash', 'RAW_faixa_etaria_por_mes(8).xlsx')
# faixa_etaria = [60,70,80,90]
# faixa_etaria_labels = ['<60','60-69','70-79','80-89','>=90']
# faixa_etaria_colunas(casos, file_name, faixa_etaria, faixa_etaria_labels)

# casos = df_casos.copy()
# file_name = join(default_output, 'dash', 'RAW_faixa_etaria_por_mes(9).xlsx')
# faixa_etaria_linhas(casos, file_name, faixa_etaria, faixa_etaria_labels)

In [ ]:
# FAIXAS ETÁRIAS DE ADOLESCENTES:

# # FAIXA ETÁRIA 11:
# file_name = join(default_output, 'dash', 'RAW_faixa_etaria_por_mes(11.1).xlsx')
# faixa_etaria = [5,10,15,20]
# faixa_etaria_labels = ['0-4','5-9','10-14','15-19','>=20']
# faixa_etaria_colunas(casos, file_name, faixa_etaria, faixa_etaria_labels)

# file_name = join(default_output, 'dash', 'RAW_faixa_etaria_por_mes(11.2).xlsx')
# faixa_etaria_linhas(casos, file_name, faixa_etaria, faixa_etaria_labels)

# # FAIXA ETÁRIA 12:
# file_name = join(default_output, 'dash', 'RAW_faixa_etaria_por_mes(12.1).xlsx')
# faixa_etaria = [15,30]
# faixa_etaria_labels = ['<15','15-29','>=30']
# faixa_etaria_linhas(casos, file_name, faixa_etaria, faixa_etaria_labels)

# file_name = join(default_output, 'dash', 'RAW_faixa_etaria_por_mes(12.2).xlsx')
# faixa_etaria_linhas(casos, file_name, faixa_etaria, faixa_etaria_labels)

In [ ]:
casos = df_casos.copy()
obitos = casos.loc[casos['obito_covid-19'] == 'SIM']
obitos['data_obito'] = pd.to_datetime(obitos['data_obito'], format='%Y-%m-%d', errors='coerce')
obitos['ano_obito'] = obitos['data_obito'].dt.year
obitos['mes_obito'] = obitos['data_obito'].dt.month
media_idades = obitos.groupby(['ano_obito', 'mes_obito'])[['idade_original']].mean().reset_index().rename(columns={'idade_original':'média'})
media_idades.to_excel(join(default_output, 'dash', 'media_idades_mes.xlsx'), index=False)

In [ ]:
# casos = df_casos.copy()
# file_name = join(default_output, 'dash', 'faixa_etaria_por_criancas.xlsx')
# faixa_etaria = [3,6,9,12,15,18]
# faixa_etaria_labels = ['0-2','3-5','6-8','9-11','12-14','15-17','>=18']
# faixa_etaria_linhas(casos, file_name, faixa_etaria, faixa_etaria_labels)